In [ ]:
# this is the script for predting the building energy consumption 
# our strategy is : 
   ##  1. Clear and careful data cleanning and data feature (seperate job for liubo), aims to clean the data and add as many useful features as possible.
   ##  2. Ensemble training models for final prediction; split the training dataset based on four catergories: 
            # a. building stories (let 25 as the building story threshold, can be discussed)
            # b. HACV system type
            # c. meter types
            # d. time span (before 2017 and after 2017)
          # According to different combination of dataset, we can create 21 different type models, for each model, we use 3 differet type of ML approaches for training, so in total, we will have 21*3 = 63 models for final selection and ensemble
   ## 3. Model validation appraoch
           # choose number 5 building as the validation model since this building info is close to target building with 29 stroies and same HACV system.  

In [32]:
# this script is to do the data cleaning, visualization and adding new features
import pandas as pd 
import numpy as np
import matplotlib
import os, sys, gc, warnings, random, datetime, math
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
matplotlib.rcParams['figure.dpi'] = 100
sns.set()
%matplotlib inline
print(os.getcwd())

/Users/nanzhang/Documents/GitHub/energyconsumption
1.0.3


In [27]:
train = pd.read_pickle("./data/0516_data.pkl")
test  = pd.read_csv("./data/result.csv")

In [28]:
# convert the datatype
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [29]:
train  =  reduce_mem_usage(train)
train.drop(columns='Unnamed: 0')

Mem. usage decreased to 52.95 Mb (56.6% reduction)


In [3]:
from dataclasses import dataclass
from typing import List
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class ModelWrapper:
    name: str
    model: 'typing.Any'
    hyper: dict
    path: str
    _estimator: 'typing.Any' = None
        
    @property
    def estimator(self):
        if self._estimator is None:
            self._estimator = self.model(**self.hyper)

        return self._estimator  

In [4]:
import lightgbm as lgb
from sklearn import ensemble
from sklearn import linear_model

# build the logic for constructing necessary models
SEED = 42
config = [
   ('H', 'Stair1', 25),
   ('C', 'HVACType', '集中式全空'),
   ('Y', 'Time', [2017,1,1])
]
config1=[0,1,2]
lgb_params = {
            'objective':'binary',
            'boosting_type':'gbdt',
            'metric':'auc',
            'n_jobs':-1,
            'learning_rate':0.01,
            'num_leaves': 2**8,
            'max_depth':-1,
            'tree_learner':'serial',
            'colsample_bytree': 0.7,
            'subsample_freq':1,
            'subsample':0.7,
            'n_estimators':80000,
            'max_bin':255,
            'verbose':-1,
            'seed': SEED,
            'early_stopping_rounds':100, 
                } 
MODELS = [
    ModelWrapper(name='log', model=linear_model.LogisticRegression, hyper={'max_iter': 100, 'penalty': 'l2', 'dual': False, 
                                               'solver': 'liblinear', 'C': 0.5}, path = ''),
    ModelWrapper(name='rfc-1', model=ensemble.RandomForestClassifier, hyper={'n_estimators': 2}, path = ''),
    ModelWrapper(name='lgb', model=lgb, hyper=lgb_params, path = ''),    
]

In [4]:
def combination_k(s, k):
    if k == 0: return ['']
    subletters = []
    # 此处涉及到一个 python 遍历循环的特点：当遍历的对象为空（列表，字符串...）时，循环不会被执行，range(0) 也是一样
    for i in range(len(s)):
        for letter in combination_k(s[i+1:], k-1):
            subletters += [str(s[i]) + str(letter)]
    return subletters

['01', '02', '12']


In [ ]:
# constructing loop for filter differet train set for train
validat_set = train[train['BuildingID'] == 5]
train = train[train['BuildingID'] != 5]
record =[]
for i in range (1,5):
    combs = combination_k(config1, i)
    for com in combs:
        logger.info(f'processing the train data with combine of {com}')
        train_set  = fliter_data(train, com)
        # output the best model path 
        Best_paths = train_model(train_set, MODELS, com)
         # output the validation mse result under best model
        val_result = valida_model(Best_paths, validat_set)
        record.append({com: [Best_paths, val_result]})
print(record)

In [80]:
def fliter_data(train, com):
    for con in com:
        conf =  config[int(con)]
        train = do_filter_data(train, conf)
    return train

def do_filter_data(train, conf):
    if conf[0] == 'H':
        train = train[train[conf[1] >= conf[2]]]
    elif conf[0] == 'C':
        train = train[train[conf[1] == conf[2]]]
    elif conf[0] == 'Y':
        train = train[train[conf[1] >= pd.Timestamp(conf[2][0], conf[2][1], conf[2][2])]]
    return train

In [ ]:
from sklearn import preprocessing
from typing import Tuple
from sklearn.externals import joblib
# here we use three models for training C
def train_model(train_set, models: List[ModelWrapper], com):

    Types =['W/Q', 'Q', 'W']
    best =[]
    for Type in Types:

        if Type =='W/Q':
            train_feed = train_set
        else:
            train_feed = train_set[train_set['Type']= Type]

        X, y, feature_names = scale_features(train_feed)

        best_modelwrapper = find_best_model(X, y, models)

        path = fit_and_save(X, y, best_modelwrapper, com, Type)

        best.append(path)

        logger.info(f'''best model for Type ={Type} and data type {com} is {best_modelwrapper.name}''')

    return best

def scale_features(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, list]:
    """ 
        1. Scale features - necessary step for some models
        2. Extract the X & y matrices
    """
    logger.info('scaling features... ')
    # Extract arrays
    y = df['Record'].values
    
    # lab_enc = preprocessing.LabelEncoder()
    # y = lab_enc.fit_transform(y)
    
    X = df.drop(['Record', 'Time'], axis='columns').values
    feature_names = df.drop('Record', axis='columns').columns.tolist()
    
    # Scale
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)

    logger.info(X.shape)

    return X, y, feature_names

def find_best_model(X: np.ndarray, y: np.ndarray, models: List[ModelWrapper]) -> ModelWrapper:
    """ Completely new function
            1. Iterate each model defined in the MODELS list
            2. Get the Cross Validated AUC for it
            3. Select the best model based on AUC, and returns it
    """
    # test all models
    results = []
    for modelwrapper in models:
        logger.info(f'*** Training {modelwrapper.name} ***')
        scores = cross_validation(X, y, modelwrapper.estimator)
        results.append([scores.mean(), modelwrapper])

    # find best model
    best_modelwrapper = sorted(results, key=lambda k: k[0], reverse=True)[0][1]
    logger.info(f'*** Best model is {best_modelwrapper.name} ***')
    return best_modelwrapper

def fit_and_save(X: np.ndarray, y: np.ndarray, model: ModelWrapper, com, Type) -> ModelWrapper:
    """
        Slight adapation of the previous code to receive the ModelWrapper
    """
    logger.info('training model')
    model.estimator.fit(X, y)

    path = f'models/{Type}_{com}_{model.name}.model' ## MODIFIED TO USE THE ModelWrapper Class
    logger.info(f'saving model in {path}')
    joblib.dump(model.estimator, path)
    return path

In [ ]:
from sklearn import model_selection

"""
    root_mean_squared_log_error
    
    from sklearn.metrics import mean_squared_log_error
    np.sqrt(mean_squared_log_error(y_test, predictions))
    
"""

def cross_validation(X, y, estimator, n_jobs: int=1):
    """
        Cross validation (CV) is a common technique used to test the stability of a prediction.
        CV Warns us if we're overfitting
        In other words, CV help us select the model which will perform best on unseen data.
    """
    logger.info('Cross Validation')
    # cross_validation
    cv = model_selection.ShuffleSplit(n_splits=5,
                                      test_size=0.2,
                                      random_state=0
                                      )


    scores = model_selection.cross_val_score(estimator,
                                                 X,
                                                 y,
                                                 cv=cv,
                                                 scoring='mean_squared_error',
                                                 n_jobs=n_jobs
                                                 )
    scores = np.sqrt(scores)
    logger.info('CV performance:')
    logger.info(
        "mean_squared_log_error: {:.2f} % (+/- {:.2f} )".format(scores.mean() * 100,
                                                        scores.std() * 2 * 100
                                                  )
                )
    return scores

In [ ]:
def valida_model(Best_paths, validat_set, com):

    Types =['W/Q', 'Q', 'W']
    val_results =[]
    for Type in Types:

        model_path =  [for path in Best_paths if Type in path]

        if Type =='W/Q':
            train_feed = validat_set
            
        else:
            train_feed = validat_set[train_set['Type']= Type]

        X, y, feature_names = scale_features(train_feed)

        result = load_and_predict(X, y, model_path)

        val_results.append(path)

        logger.info(f'''The validation results for Type ={Type} and data type {com} is {result}''')

    return val_results


def load_and_predict(X, y, model_path):
    estimator = joblib.load(model_path[0])
    pre_y = estimator.predict(X)
    result = rmse(pre_y, y)
    return result


def rmse(predictions, targets):

    differences = predictions - targets                       #the DIFFERENCEs.

    differences_squared = differences ** 2                    #the SQUAREs of ^

    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^

    rmse_val = np.sqrt(mean_of_differences_squared)/ targets.mean()  #ROOT of ^/ MEAN of target

    return rmse_val 